In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
from scipy import signal
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.distributions import Categorical

In [ ]:
torch.manual_seed(20)

In [ ]:
df = pd.read_csv('aapl.us.txt').iloc[6000:8001]
apl_open = np.array(df['Open'])
apl_close = np.array(df['Close'])

In [ ]:
apl_open = signal.detrend(apl_open)
apl_close = signal.detrend(apl_close)
print(apl_open.min(), apl_close.min())

In [ ]:
apl_open += (-apl_open.min() + 1)
apl_close += (-apl_close.min() + 1)

In [ ]:
print(apl_open.min(), apl_close.min())

In [ ]:
apl_open_orig = np.array(df['Open'])
apl_close_orig = np.array(df['Close'])

In [ ]:
class Environment:
    def __init__(self, starting_cash=200, randomize_cash=0, starting_shares=0,
                 randomize_shares=0, max_stride=5, series_length=200,
                 starting_point=0, inaction_penalty=0):
        self.starting_cash = starting_cash
        self.randomize_cash = randomize_cash
        self.starting_shares = starting_shares
        self.randomize_shares = randomize_shares
        
        self.starting_cash = max(int(np.random.normal(self.starting_cash, self.randomize_cash)), 0)
        self.series_length = series_length
        self.starting_point = starting_point
        self.cur_timestep = self.starting_point
        
        self.state = torch.FloatTensor(torch.zeros(5)).cuda()
        self.state[0] = max(int(np.random.normal(self.starting_shares, self.randomize_shares)), 0)
        self.state[1] = self.starting_cash
        self.state[2] = apl_open[self.cur_timestep]
        self.starting_portfolio_value = self.portfolio_value()
        self.state[3] = self.starting_portfolio_value
        self.state[4] = self.five_day_window()
        
        self.max_stride = max_stride
        self.stride = self.max_stride
        
        self.done = False
        self.inaction_penalty = inaction_penalty
        
    def portfolio_value(self):
        return ((self.state[0] * apl_close[self.cur_timestep]) + self.state[1])
    
    def next_opening_price(self):
        next_timestep = self.cur_timestep + self.stride
        return apl_open[next_timestep]
    
    def five_day_window(self):
        step = self.cur_timestep
        if step < 5:
            return apl_open[0]
        apl5 = apl_open[step-5:step].mean()
        return apl5
    
    def step(self, action):
        action = [action, 1]
        cur_timestep = self.cur_timestep
        ts_left = self.series_length - (cur_timestep - self.starting_point)
        retval = None
        cur_value = self.portfolio_value()
        gain = cur_value - self.starting_portfolio_value
        
        if cur_timestep >= self.starting_point + self.series_length * self.stride:
            next_opening = self.next_opening_price()
            next_five_day = self.five_day_window()
            new_state = [self.state[0], self.state[1], next_opening, cur_value, next_five_day]
            self.state = new_state
            return new_state, cur_value + gain, True, {'msg': 'done'}
        
        if action[0] == 2:
            next_opening = self.next_opening_price()
            next_five_day = self.five_day_window()
            new_state = [self.state[0], self.state[1], next_opening, cur_value, next_five_day]
            self.state = new_state
            retval = new_state, gain - self.inaction_penalty - ts_left, False, {'msg': 'nothing'}
            
        if action[0] == 0:
            if action[1] * apl_open[cur_timestep] > self.state[1]:
                next_opening = self.next_opening_price()
                next_five_day = self.five_day_window()
                new_state = [self.state[0], self.state[1], next_opening, cur_value, next_five_day]
                self.state = new_state
                retval = new_state, -ts_left + gain / 2, True, {'msg': 'bankrupt'}
            else:
                apl_shares = self.state[0] + action[1]
                cash_spent = action[1] * apl_open[cur_timestep] * 1.1
                next_opening = self.next_opening_price()
                next_five_day = self.five_day_window()
                new_state = [apl_shares, self.state[1] - cash_spent, next_opening, cur_value, next_five_day]
                self.state = new_state
                retval = new_state, gain + self.inaction_penalty - ts_left, False, {'msg': 'bought stocks'}
                
        if action[0] == 1:
            if action[1] > self.state[0]:
                next_opening = self.next_opening_price()
                next_five_day = self.five_day_window()
                new_state = [self.state[0], self.state[1], next_opening, cur_value, next_five_day]
                self.state = new_state
                retval = new_state, -ts_left + gain / 2, True, {'msg': 'sold more than available'}
            else:
                apl_shares = self.state[0] - action[1]
                cash_gained = action[1] * apl_open[cur_timestep] * 0.9
                next_opening = self.next_opening_price()
                next_five_day = self.five_day_window()
                new_state = [apl_shares, self.state[1] + cash_gained, next_opening, cur_value, next_five_day]
                self.state = new_state
                retval = new_state, gain + self.inaction_penalty - ts_left, False, {'msg': 'sold stocks'}
                
        self.cur_timestep += self.stride
        return retval
    
    def reset(self):
        self.starting_cash = max(int(np.random.normal(self.starting_cash, self.randomize_cash)), 0.)
        self.cur_timestep = self.starting_point
        
        self.state[0] = max(int(np.random.normal(self.starting_shares, self.randomize_shares)), 0)
        self.state[1] = self.starting_cash
        self.state[2] = apl_open[self.cur_timestep]
        self.state[3] = self.starting_portfolio_value
        self.state[4] = self.five_day_window()
        
        self.done = False
        return self.state

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.input_layer = nn.Linear(5, 128)
        self.hidden_1 = nn.Linear(128, 128)
        self.hidden_2 = nn.Linear(32, 31)
        self.hidden_state = torch.tensor(torch.zeros(2, 1, 32)).cuda()
        self.rnn = nn.GRU(128, 32, 2)
        self.action_head = nn.Linear(31, 3)
        self.value_head = nn.Linear(31, 1)
        self.saved_actions = []
        self.rewards = []
        
    def reset_hidden(self):
        self.hidden_state = torch.tensor(torch.zeros(2, 1, 32)).cuda()
        
    def forward(self, x):
        x = torch.tensor(x).cuda()
        x = torch.sigmoid(self.input_layer(x))
        x = torch.tanh(self.hidden_1(x))
        x, self.hidden_state = self.rnn(x.view(1, -1, 128), self.hidden_state.data)
        x = F.relu(self.hidden_2(x.squeeze()))
        action_scores = self.action_head(x)
        state_values = self.value_head(x)
        return F.softmax(action_scores, dim=-1), state_values
    
    def act(self, state):
        probs, state_value = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        if action == 1 and state[0] < 1:
            action = torch.LongTensor([2]).squeeze().cuda()
        self.saved_actions.append((m.log_prob(action), state_value))
        return action.item()

In [ ]:
env = Environment(starting_cash=1000, randomize_cash=100, starting_shares=100, 
                  randomize_shares=10, max_stride=4, series_length=499)
model = Policy().cuda()
optimizer = optim.Adam(model.parameters(), lr=4e-4)

In [ ]:
env.reset()
del model.rewards[:]
del model.saved_actions[:]

In [ ]:
gamma = 0.8
log_interval = 40
running_reward = 0

In [ ]:
def finish_episode():
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    rewards = []
    for r in model.rewards[::-1]:
        R = r + (gamma * R)
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    epsilon = (torch.rand(1) / 1e4) - 5e-5
    rewards += epsilon
    
    for (log_prob, value), r in zip(saved_actions, rewards):
        reward = torch.tensor(r - value.item()).cuda()
        policy_losses.append(-log_prob * reward)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([r]).cuda()))
        
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    loss = torch.clamp(loss, -1e-5, 1e5)
    loss.backward()
    optimizer.step()
    del model.rewards[:]
    del model.saved_actions[:]

In [ ]:
for episode in range(0, 4000):
    state = env.reset()
    reward = 0
    done = False
    msg = None
    while not done:
        action = model.act(state)
        state, reward, done, msg = env.step(action)
        model.rewards.append(reward)
        if done:
            break
    running_reward = running_reward * (1 - 1 / log_interval) + reward * 1 / log_interval
    finish_episode()
    
    if msg['msg'] == 'done' and running_reward > 300 and env.portfolio_value() > env.starting_portfolio_value:
        print('Early stop: ', int(reward))
        break
    if episode % log_interval == 0:
        print('Episode {}: start {:.1f}, end {:.1f} because {} @ t={}, last reward {:.1f}, running reward {:.1f}' \
              .format(episode, env.starting_portfolio_value, env.portfolio_value(), msg['msg'],
                      env.cur_timestep, reward, running_reward))

In [ ]:
total_rewards = 0
total_profits = 0
failed_goes = 0
num_goes = 50

for j in range(num_goes):
    env.reset()
    reward_this_go = -1e8
    for i in range(0, env.series_length + 1):
        action = model.act(env.state)
        next_state, reward, done, msg = env.step(action)
        if msg['msg'] == 'done':
            reward_this_go = env.portfolio_value()
            break
        if done:
            break
    total_profits += (env.portfolio_value() - env.starting_portfolio_value) / env.starting_portfolio_value
    if reward_this_go == -1e8:
        failed_goes += 1
    else:
        total_rewards += reward_this_go

if failed_goes == num_goes:
    print('Failed all!')
else:
    print('Failed goes: {}/{}, Average reward per successful game: {}'.format(
        failed_goes, num_goes, total_rewards / (num_goes - failed_goes)))
    print('Average profit per game: ', total_profits / num_goes)
    print('Average profit per finished game: ', total_profits / (num_goes - failed_goes))

In [ ]:
print('Starting portfolio value ', env.portfolio_value())
for i in range(0, env.series_length + 1):
    action = model.act(env.state)
    next_state, reward, done, msg = env.step(action)
    if msg['msg'] == 'bankrupted self':
        print('Bankrupted self by 1')
        break
    if msg['msg'] == 'sold more than have':
        print('Sold more than have by 1')
        break
    print('{}, have {} stocks and {} cash'.format(msg['msg'], next_state[0], next_state[1]))
    if msg['msg'] == 'done':
        print(next_state, reward)
        print('Total portfolio value ', env.portfolio_value())
        break

In [ ]:
env.reset()
complete_game = False

In [ ]:
while not complete_game:
    bought_apl_at = []
    sold_apl_at = []
    bought_apl_at_orig = []
    sold_apl_at_orig = []
    nothing_at = []
    b_action_times = []
    s_action_times = []
    n_action_times = []
    starting_val = env.starting_portfolio_value
    print('Starting portfolio value: {}'.format(starting_val))
    for i in range(0, env.series_length + 1):
        action = model.act(env.state)
        if action == 0:
            bought_apl_at.append(apl_open[env.cur_timestep])
            bought_apl_at_orig.append(apl_open_orig[env.cur_timestep])
            b_action_times.append(env.cur_timestep)
        if action == 1:
            sold_apl_at.append(apl_close[env.cur_timestep])
            sold_apl_at_orig.append(apl_close_orig[env.cur_timestep])
            s_action_times.append(env.cur_timestep)
        if action == 2:
            nothing_at.append(0)
            n_action_times.append(env.cur_timestep)
        next_state, reward, done, msg = env.step(action)
        if msg['msg'] == 'bankrupted self':
            env.reset()
            break
        if msg['msg'] == 'sold more than have':
            env.reset()
            break
        if msg['msg'] == 'done':
            print('{}, have {} stocks and {} cash'.format(msg['msg'], next_state[0], next_state[1]))
            val = env.portfolio_value()
            print('Finished portfolio value {}'.format(val))
            complete_game = True
            env.reset()
            break

In [ ]:
plt.figure(1, figsize=(14, 5))
plt.plot(range(0, len(apl_open)), apl_open)
plt.plot(b_action_times, bought_apl_at, 'ro')
plt.plot(s_action_times, sold_apl_at, 'go')
plt.plot(n_action_times, nothing_at, 'yx')

In [ ]:
plt.figure(1, figsize=(14, 5))
plt.plot(range(0, len(apl_open_orig)), apl_open_orig)
plt.plot(b_action_times, bought_apl_at_orig, 'ro')
plt.plot(s_action_times, sold_apl_at_orig, 'go')
plt.plot(n_action_times, nothing_at, 'yx')